In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process.kernels import Matern, RBF
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, ExpSineSquared
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse

In [46]:
df = pd.read_csv('data/gridsearch.matern.csv', sep=',')
df.head()

,category,learner,query_strategy,kernel,length_scale,nu,n_learners,n_initial,n_queries,r2
0,Committee,Gaussian Process,max_std_sampling,"Matern(length_scale=0.1, nu=1.5)",0.100000,1.5,2,10,100,-14.425781
1,Committee,Gaussian Process,max_std_sampling,"Matern(length_scale=0.644, nu=1.5)",0.644444,1.5,2,10,100,0.325172
2,Committee,Gaussian Process,max_std_sampling,"Matern(length_scale=1.19, nu=1.5)",1.188889,1.5,2,10,100,0.354015
3,Committee,Gaussian Process,max_std_sampling,"Matern(length_scale=1.73, nu=1.5)",1.733333,1.5,2,10,100,0.289640
4,Committee,Gaussian Process,max_std_sampling,"Matern(length_scale=2.28, nu=1.5)",2.277778,1.5,2,10,100,0.294963


In [47]:
df = df[df['r2']>0]

In [48]:

df = df.drop(['category', 'learner', 'query_strategy','kernel'],axis=1)
y = df['r2']
X = df.drop(['r2'],axis=1)

scaler = StandardScaler()
X2 = scaler.fit_transform(X)

In [49]:
reg = LinearRegression().fit(X2, y)




In [50]:
np.round(reg.coef_,4)



array([-0.0023,  0.    ,  0.0394,  0.0446,  0.    ])

In [6]:
data = pd.read_csv('data/hw3_data.csv', delimiter=',',header=0)
print(data.shape)
data.head(5)

# create separate columns for each amino acid
for i in range(9):
    colname='seq'+str(i)
    data[colname] = [x[i] for x in data['seq']]
    
X_df = data.drop(['pIC50','id','allele', 'seq'],axis=1)
y = data['pIC50']
print(X_df.shape)
print(y.shape)

enc = OneHotEncoder(handle_unknown='ignore')
X_enc = enc.fit_transform(X_df)
print(type(X_enc))
print(X_enc.shape)

# convert to numpy array
X_pool=sparse.csr_matrix.toarray(X_enc)
print(type(X_pool))
print(X_pool.shape)
y_pool = y.to_numpy()
print(y_pool.shape)

(9051, 4)
(9051, 9)
(9051,)
<class 'scipy.sparse.csr.csr_matrix'>
(9051, 180)
<class 'numpy.ndarray'>
(9051, 180)
(9051,)


In [ ]:
# Fit KernelRidge with parameter selection based on 5-fold cross validation
param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3],
              "kernel": [ExpSineSquared(l, p)
                         for l in np.logspace(-2, 2, 10)
                         for p in np.logspace(0, 2, 10)]}
kr = GridSearchCV(KernelRidge(), param_grid=param_grid)
kr.fit(X_pool, y_pool)

/Users/ryanqnelson/opt/anaconda3/envs/cmu/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/Users/ryanqnelson/opt/anaconda3/envs/cmu/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
